# STA

## Data Gathering

In [13]:
import requests
import pandas as pd
import json


result = requests.get("https://registrar-apps.ucdavis.edu/courses/search/course_search_results.cfm", params = {
    'subject': 'STA',
    'termCode': '202401'
})

#IDK if i'm getting every row as well

In [14]:
from bs4 import BeautifulSoup
html_content = result.text
soup = BeautifulSoup(html_content, 'html.parser')
row = soup.find_all('tr', {'bgcolor':'#EBEFF5'})

data = pd.DataFrame()

pd.set_option('display.max_rows', None)

#So the way the registrar does it, they have one row for lec & one for disc. My code takes whichever comes first. Sections are split by crn
#So for my purposes, idrc. It's actually better maybe so it doesn't double count
for section in row:
    row_data = []
    for cell in section.find_all('td'):
        cell_data = cell.get_text(strip=True)
        row_data.append(cell_data)
    data = pd.concat([data, pd.DataFrame(row_data).T])

#Also the bottom gets really screwy but I might just do upper div and not grad
data

,0,1,2,3,4,5
0,"425898:00 - 8:50 AM, T",STA 013OLSON 147,A012/0/3,Elementary StatisticsSE•QLSE,"Whitener, J4.0",view42589
0,"425909:00 - 9:50 AM, MWF",STA 013ROCK 194,A020/0/3,Elementary StatisticsSE•QLSE,"Whitener, J4.0",view42590
0,"425919:00 - 9:50 AM, MWF",STA 013ROCK 194,A031/0/4,Elementary StatisticsSE•QLSE,"Whitener, J4.0",view42591
0,"425929:00 - 9:50 AM, MWF",STA 013ROCK 194,A040/0/3,Elementary StatisticsSE•QLSE,"Whitener, J4.0",view42592
0,"425938:00 - 8:50 AM, R",STA 013OLSON 147,B010/0/4,Elementary StatisticsSE•QLSE,"Whitener, J4.0",view42593
0,"425949:00 - 9:50 AM, R",STA 013OLSON 223,B021/0/4,Elementary StatisticsSE•QLSE,"Whitener, J4.0",view42594
0,"4259511:00 - 11:50 AM, MWF",STA 013ROCK 194,B033/0/5,Elementary StatisticsSE•QLSE,"Whitener, J4.0",view42595
0,"4259611:00 - 11:50 AM, R",STA 013OLSON 250,B041/0/5,Elementary StatisticsSE•QLSE,"Whitener, J4.0",view42596
0,"4259712:10 - 1:00 PM, R",STA 013HART 1130,B050/0/5,Elementary StatisticsSE•QLSE,"Whitener, J4.0",view42597
0,"425981:10 - 2:00 PM, R",STA 013HART 1130,B060/0/5,Elementary StatisticsSE•QLSE,"Whitener, J4.0",view42598


## Data Cleaning

### Initial

In [15]:
#Take out na rows
data_clean = data.dropna()

#Reset index
data_clean = data_clean.reset_index(drop=True)

#data_clean

### Inner Cleaning

#### Row 0

*Note: you can only run the code once*

In [16]:
#Take out course numbers from time
data_clean.iloc[:,0] = data_clean.iloc[:,0].str[5:]
#Take out days
data_clean.iloc[:,0] = data_clean.iloc[:,0].str.split(",").str.get(0)

#data_clean

#### Row 1

*WARNING THIS MESSES THINGS UP FOR COURSES LIKE STA 194HB*

In [17]:
#Take only course name bc we have course code
data_clean.iloc[:,1] = data_clean.iloc[:,1].str[:8] #Get course number but also first letter after for courses like sta 141b
#Now get rid of any course who's letter isn't a,b, or c. Bc if not, it's remnants from hall

#So this still keeps some hall names. What I can try to do is either make hall names lowercase. Or I can try to have a list of hall names to remove
data_clean.iloc[:,1] = data_clean.iloc[:,1].str.replace('[^STABC\d]','',regex=True)

#Add a space after STA, you can also probably take out any S here... or maybe you could've done this first then take out the hall name
data_clean.iloc[:,1] = data_clean.iloc[:,1].str[:3] + " " + data_clean.iloc[:,1].str[3:].replace('[^ABC\d]', '', regex=True)

#data_clean

#### Row 3

In [18]:
#Get rid of everything after •
data_clean.iloc[:,3] = data_clean.iloc[:,3].str.split("•").str.get(0)
#Get rid of any other GE's or "No GE Credit"
data_clean.iloc[:,3] = data_clean.iloc[:,3].str[:-2] #Gets rid of last 2 elements. Fine bc all titles have SOMETHING there
data_clean.iloc[:,3] = data_clean.iloc[:,3].str.split('No ').str.get(0)

#data_clean

#### Row 5

In [19]:
#Clean with only course numbers
data_clean.iloc[:,5] = data_clean.iloc[:,5].str.replace(r'[view]', '', regex=True)

#data_clean

### Column Manipulation

#### Row 2

In [20]:
#Split section & availability into new column
#Theoretically, the course numbers should all be 3 digits long aside from some exceptions.
#Maybe I can restring so if there's one without the required total # of digits, just add some 0's. I can't do that because there are some
# Courses that have single digits

data_clean["Availability"] = data_clean.iloc[:,2].str[3:]
data_clean.iloc[:,2] = data_clean.iloc[:,2].str[:3]

#data_clean

#### Row 4

In [21]:
#Split  prof & units
data_clean["Units"] = data_clean.iloc[:,4].str[-3:]
data_clean.iloc[:,4] = data_clean.iloc[:,4].str[:-3]

#data_clean

#### Row 6

In [10]:
#Remove

In [22]:
#data_clean = data_clean.drop(columns=[6])

#data_clean

KeyError: '[6] not found in axis'

### Finishing Touches

In [ ]:
#Remaining
#Col 1, some class codes have an S... also this is very specified to stats. In spanish for example, you have 100S or 100N
#Actually in the future, i think for each <td> element, you can split it based on the text and the <ed> bc it looks like the html separates
#Course from building. which would've been good to know iat the beginning

#Automate at certain times in day... Then I'd need a server?

#Ah fudge I just realized, if the time can be either from discussion or lecture, it's gonna be scuffed. Luckily I don't plan on doing
#   too much with time rn but I should try to change that eventually but I don't think the registrar differentiates

#### Renaming

In [24]:
column_names = ['Time?', 'Course', 'Section', 'Course Name', 'Prof', 'CRN', 'Availability', 'Units']
data_clean.columns = column_names

#data_clean

#### Drop Non-Standard Undergrad Classes

In [25]:
data_clean = data_clean.drop(data_clean.index[74:])
data_clean = data_clean.drop(21)

#data_clean

#### Reset Index

In [26]:
data_clean = data_clean.set_index('CRN')
#data_clean

## Reformatting Data Frame to Specific Use of calculating number of seats per course

In [27]:
data_clean

,Time?,Course,Section,Course Name,Prof,Availability,Units
CRN,,,,,,,
42589,8:00 - 8:50 AM,STA 013,A01,Elementary Statistics,"Whitener, J",2/0/3,4.0
42590,9:00 - 9:50 AM,STA 013,A02,Elementary Statistics,"Whitener, J",0/0/3,4.0
42591,9:00 - 9:50 AM,STA 013,A03,Elementary Statistics,"Whitener, J",1/0/4,4.0
42592,9:00 - 9:50 AM,STA 013,A04,Elementary Statistics,"Whitener, J",0/0/3,4.0
42593,8:00 - 8:50 AM,STA 013,B01,Elementary Statistics,"Whitener, J",0/0/4,4.0
42594,9:00 - 9:50 AM,STA 013,B02,Elementary Statistics,"Whitener, J",1/0/4,4.0
42595,11:00 - 11:50 AM,STA 013,B03,Elementary Statistics,"Whitener, J",3/0/5,4.0
42596,11:00 - 11:50 AM,STA 013,B04,Elementary Statistics,"Whitener, J",1/0/5,4.0
42597,12:10 - 1:00 PM,STA 013,B05,Elementary Statistics,"Whitener, J",0/0/5,4.0


In [28]:
import datetime
import time
date = datetime.date.today()
current_time = time.strftime('%H:%M')
#create a new data set with just the course as the index, the course name, units, and the sum of availability

#first add a new column for just the open seats (def by open - waitlist)
data_clean[f"{date}, {current_time}"] = data_clean["Availability"].str.split("/").str[0].astype(int) - data_clean["Availability"].str.split("/").str[-1].astype(int)

#Create new dataset of aggregates
data_spec = pd.DataFrame(columns = ['Course', f"{date}, {current_time}"])
data_spec['Course'] = data_clean['Course'].unique()
#data_spec['Course Name'] = data_clean['Course Name'].unique()
data_spec[f"{date}, {current_time}"] = list(data_clean.groupby(['Course'])[f"{date}, {current_time}"].sum())
data_spec = data_spec.set_index('Course')
data_spec

#Can add profs too

,"2023-12-20, 04:33"
Course,
STA 013,-44
STA 032,39
STA 035A,29
STA 035B,18
STA 100,-19
STA 103,38
STA 104,47
STA 106,-12
STA 108,-22


## Adding Data to CSV

**In function**

In [ ]:

#Take csv to data frame
#Add og data frame to the csv data frame, based on index, & create new column based on current date (maybe get rid of course name for index)
#     or combine names
#put new data frame to csv



#Create csv: 
#fhand = open("schedBuild.csv", 'w')
#fhand.close()
#data_spec.to_csv('schedBuild.csv', index=True)

#Can't create a csv out of nowhere, so this is adding columns
og_csv = pd.read_csv('schedBuild.csv')
new_csv = pd.merge(og_csv, data_spec, on='Course')
new_csv.to_csv('schedBuild.csv', index=False)

## Function Conversion

In [1]:
import os
import datetime
import time
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup

def get_data(subject):
    result = requests.get("https://registrar-apps.ucdavis.edu/courses/search/course_search_results.cfm", params = {
        'subject': subject,
        'termCode': '202401' #Assumed winter
    })
    html_content = result.text
    soup = BeautifulSoup(html_content, 'html.parser')
    row = soup.find_all('tr', {'bgcolor':'#EBEFF5'})
    data = pd.DataFrame()
    for section in row:
        row_data = []
        for cell in section.find_all('td'):
            cell_data = cell.get_text(strip=True)
            row_data.append(cell_data)
        data = pd.concat([data, pd.DataFrame(row_data).T])
    return data

def clean_data(data, subject):
    #I'm gonna need a better way to organize the courses separate from building. Let's see  if I can parse it during data gathering
    data_clean = data.dropna()
    data_clean = data_clean.reset_index(drop=True)

    #HERE I DROP NON STANDARD CLASSES But it's difficult
    #To drop grad classes, i'll take out anything who's str index at 5 is 2, ig for honors like stats we could do by 090? Let's go one at a time
    data_clean = data_clean[data_clean.iloc[:,1].str[4] != '2']
    #data_clean = data_clean.drop(data_clean.index[74:])
    #data_clean = data_clean.drop(21)

    #Col 0
    data_clean.iloc[:,0] = data_clean.iloc[:,0].str[5:]
    data_clean.iloc[:,0] = data_clean.iloc[:,0].str.split(",").str.get(0)

    #Col 1
    data_clean.iloc[:,1] = data_clean.iloc[:,1].str[:8]
    included_letters = None
    if subject=='STA':
        included_letters = 'ABC'
    elif subject == 'MAT':
        included_letters = 'ABCD'
    elif subject == 'ECN':
        included_letters = 'AB'
    elif subject == 'SPA':
        included_letters = 'N'
    data_clean.iloc[:,1] = data_clean.iloc[:,1].str.replace(f'[^{subject}{included_letters}\d]','',regex=True)
    #This one assumes the only courses you have, have abc. Also it assumes only one letter like ecn 115b not ecn 115by
    data_clean.iloc[:,1] = data_clean.iloc[:,1].str[:3] + " " + data_clean.iloc[:,1].str[3:].replace(f'[^{included_letters}\d]', '', regex=True)

    #Col 3
    data_clean.iloc[:,3] = data_clean.iloc[:,3].str.split("•").str.get(0)
    data_clean.iloc[:,3] = data_clean.iloc[:,3].str[:-2] #Gets rid of last 2 elements. Fine bc all titles have SOMETHING there
    data_clean.iloc[:,3] = data_clean.iloc[:,3].str.split('No ').str.get(0)

    #Col 5
    data_clean.iloc[:,5] = data_clean.iloc[:,5].str.replace(r'[view]', '', regex=True)

    #Col 2
    data_clean["Availability"] = data_clean.iloc[:,2].str[3:]
    data_clean.iloc[:,2] = data_clean.iloc[:,2].str[:3]

    #Col 4
    data_clean["Units"] = data_clean.iloc[:,4].str[-3:]
    data_clean.iloc[:,4] = data_clean.iloc[:,4].str[:-3]

    #Col 6
    #data_clean = data_clean.drop(columns=[6])

    return data_clean

def format_data(data_clean):
    column_names = ['Time?', 'Course', 'Section', 'Course Name', 'Prof', 'CRN', 'Availability', 'Units']
    data_clean.columns = column_names

    data_clean = data_clean.set_index('CRN')
    date = datetime.date.today()
    current_time = time.strftime('%H:%M')
    data_clean[f"{date}, {current_time}"] = data_clean["Availability"].str.split("/").str[0].astype(int) - data_clean["Availability"].str.split("/").str[-1].astype(int)
    data_spec = pd.DataFrame(columns = ['Course', f"{date}, {current_time}"])
    data_spec['Course'] = data_clean['Course'].unique()
    data_spec[f"{date}, {current_time}"] = list(data_clean.groupby(['Course'])[f"{date}, {current_time}"].sum())
    data_spec = data_spec.set_index('Course')
    return data_spec

def update_csv(data_spec, subject):
    try:
        og_csv = pd.read_csv(f'schedBuild_{subject}.csv')
        new_csv = pd.merge(og_csv, data_spec, on='Course')
        new_csv.to_csv(f'schedBuild_{subject}.csv', index=False)
    except:
        fhand = open(f'schedBuild_{subject}.csv', 'w')
        fhand.close()
        data_spec.to_csv(f'schedBuild_{subject}.csv', index=True)
        og_csv = pd.read_csv(f'schedBuild_{subject}.csv')
        new_csv = pd.merge(og_csv, data_spec, on='Course')
        new_csv.to_csv(f'schedBuild_{subject}.csv', index=False)
        
#Issue is it might get scuffed with non standard classes but we ball. Maybe the non standard classes we can do it by course name

def schedBuild(subject):
    data = get_data(subject)
    cleaned = clean_data(data, subject)
    formatted = format_data(cleaned)
    update_csv(formatted, subject)
    os.system(f'open schedBuild_{subject}.csv') #opens file on computer


In [2]:
schedBuild('STA')

KeyError: '[6] not found in axis'

## Schedule Program

## FINAL

In [ ]:
#Possibly segregate it by prof too
#put it into function
#make it so that it works for different majors. The hard part is that some majors like spanish will have 100s and a hall beginning with s
#You could put parameters for functions defining majors and then exclude dif letters based on those parameters

#This is also only for winter quarter, doesn't take into account the past

# ECN

## Data Gathering

In [ ]:
#Turn it into a function